In [1]:
from datetime import datetime, timedelta

import pandas as pd
from google.cloud import bigquery
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor

from utils import (
    create_spreadsheet,
    share_spreadsheet,
    authenticate,
    add_worksheet_and_write_data,
)
from config import GOOGLE_APPLICATION_CREDENTIALS, SCOPES, EMAIL

In [2]:
load_dotenv()

client = bigquery.Client()

Create a list of queries with specific dates

In [3]:
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 1, 3)

current_date = start_date

queries = [
    f"SELECT * "
    f"FROM `bigquery-public-data.google_analytics_sample.ga_sessions_{current_date.strftime('%Y%m%d')}`"
    for current_date in (
        start_date + timedelta(days=i)
        for i in range((end_date - start_date).days + 1)
    )
]

Concurrently retrieve data from bigquery with list of queries

In [4]:
def execute_query(query):
    return client.query(query).to_dataframe()


def execute_queries(queries):
    with ThreadPoolExecutor(4) as executor:
        results = executor.map(execute_query, queries)

        return list(results)


dataframes = execute_queries(queries)

C:\Users\PC\Desktop\Employment\newage.agency\DanaAnalysTT\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Making dataframe from dataframes list

In [24]:
df = pd.concat(dataframes)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5387 entries, 0 to 2402
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5387 non-null   Int64 
 2   visitId               5387 non-null   Int64 
 3   visitStartTime        5387 non-null   Int64 
 4   date                  5387 non-null   object
 5   totals                5387 non-null   object
 6   trafficSource         5387 non-null   object
 7   device                5387 non-null   object
 8   geoNetwork            5387 non-null   object
 9   customDimensions      5387 non-null   object
 10  hits                  5387 non-null   object
 11  fullVisitorId         5387 non-null   object
 12  userId                0 non-null      object
 13  channelGrouping       5387 non-null   object
 14  socialEngagementType  5387 non-null   object
dtypes: Int64(4), object(11)
memory usage: 694.4

Cleaning dataframe from columns with missing values

In [25]:
df = df.drop(["visitorId", "userId"], axis=1)

Updating data types

In [ ]:
df["visitNumber"] = df["visitNumber"].astype("uint16")
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

Make dataframe samples

In [28]:
df_channel_counts = df.groupby("channelGrouping").size().reset_index()
df_group_by_fullVisitorId = df.groupby("fullVisitorId").size().reset_index()
df_group_by_socialEngagementType = (
    df.groupby("socialEngagementType").size().reset_index()
)


dataframes_to_save = {
    "channel_counts": df_channel_counts,
    "group_by_fullVisitorId": df_group_by_fullVisitorId,
    "group_by_socialEngagementType": df_group_by_socialEngagementType,
}

In [ ]:
gclient = authenticate(scopes=SCOPES, credentials=GOOGLE_APPLICATION_CREDENTIALS)

spreadsheet = create_spreadsheet(gclient, "task_1")
share_spreadsheet(spreadsheet, EMAIL)

Post dataframes to google sheets

In [30]:
with ThreadPoolExecutor(4) as executor:
    futures = []
    for sheet_name, df in dataframes_to_save.items():
        futures.append(
            executor.submit(
                add_worksheet_and_write_data, spreadsheet, df, sheet_name
            )
        )

    for future in futures:
        future.result()